<a href="https://colab.research.google.com/github/aquafire088/ZEHPRY/blob/main/51125.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qU transformers accelerate bitsandbytes trl peft datasets auto-gptq optimum

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.0/348.0 kB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 62.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.6/433.6 kB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 77.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 89.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install --upgrade protobuf==3.20.1

In [ ]:
!pip install bitsandbytes

In [1]:
import torch
import bitsandbytes
from transformers import AutoModelForCausalLM, AutoTokenizer, GPTQConfig, TrainingArguments,GenerationConfig, AutoTokenizer,BitsAndBytesConfig
from datasets import load_dataset,Dataset, DatasetDict
from sklearn.model_selection import train_test_split
from trl import SFTTrainer
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model,AutoPeftModelForCausalLM
import pandas as pd

In [2]:
base_model="TheBloke/zephyr-7B-beta-GPTQ"

In [3]:
#chargement des données
try:
    df = pd.read_csv("/content/sample_data/finel.csv")
except Exception as e:
    raise ValueError(f"Erreur lors du chargement du CSV: {str(e)}")
# Aperçu des données
print(df.head())
# Vérification des colonnes
required_columns = {'category', 'instruction', 'input', 'output'}
missing_columns = required_columns - set(df.columns)

if missing_columns:
    print(f"Colonnes manquantes détectées : {missing_columns}")
    print("Colonnes disponibles dans le DataFrame :", list(df.columns))
    raise ValueError(f"Colonnes requises manquantes : {missing_columns}")
else:
    print("Toutes les colonnes requises sont présentes dans le DataFrame.")

  category                                        instruction  \
0   Module  Listez les modules de chaque semestre. Pour Bi...   
1   Module  Quel est l'objectif du Module M19 à M24 ? Pour...   
2   Module  Listez les modules du Semestre 1. Pour Big Dat...   
3   Module  Listez les modules du Semestre 2. Pour Big Dat...   
4   Module  Listez les modules du Semestre 3. Pour Big Dat...   

                                               input  \
0  Quels sont les modules enseignés par semestre ...   
1  Que représente le stage en Semestre 4 de Big D...   
2  Quels sont les modules enseignés en Semestre 1...   
3  Quels sont les modules enseignés en Semestre 2...   
4  Quels sont les modules enseignés en Semestre 3...   

                                              output  
0   "M1 (Fondamentaux Algorithmiques Pour Big Dat...  
1  Stage en entreprise pour appliquer les connais...  
2  M1 : Fondamentaux Algorithmiques Pour Big Data...  
3  M7 : Mobile Cloud Computing, M8 : Hadoop Base ...

In [4]:
# Nettoyage des données
import numpy as np
import pandas as pd

def clean_data(value):
    """Nettoie une valeur individuelle"""
    if pd.isna(value) or value is None:
        return ""
    if isinstance(value, (np.ndarray, list)):
        return " ".join(map(str, value)) if len(value) > 0 else ""
    return str(value).strip()

# Afficher un échantillon avant nettoyage
print("\n=== AVANT NETTOYAGE ===")
print("Exemples de données avant nettoyage :")
for col in required_columns:
    print(f"\nColonne {col}:")
    print(df[col].head(3).to_markdown(tablefmt="grid"))

# Appliquer le nettoyage
for col in required_columns:
    df[col] = df[col].fillna("").apply(clean_data)

# Afficher un échantillon après nettoyage
print("\n=== APRÈS NETTOYAGE ===")
print("Exemples de données après nettoyage :")
for col in required_columns:
    print(f"\nColonne {col}:")
    print(df[col].head(3).to_markdown(tablefmt="grid"))

# Statistiques de nettoyage
print("\n=== STATISTIQUES ===")
print(f"Nombre total de lignes : {len(df)}")
for col in required_columns:
    empty_count = (df[col] == "").sum()
    print(f"Vides dans {col}: {empty_count} ({empty_count/len(df):.1%})")


=== AVANT NETTOYAGE ===
Exemples de données avant nettoyage :

Colonne input:
+----+--------------------------------------------------------------------------------+
|    | input                                                                          |
+====+================================================================================+
|  0 | Quels sont les modules enseignés par semestre De Big Data et Cloud Computing?  |
+----+--------------------------------------------------------------------------------+
|  1 | Que représente le stage en Semestre 4 de Big Data et Cloud Computing?          |
+----+--------------------------------------------------------------------------------+
|  2 | Quels sont les modules enseignés en Semestre 1 de Big Data et Cloud Computing? |
+----+--------------------------------------------------------------------------------+

Colonne category:
+----+------------+
|    | category   |
+====+============+
|  0 | Module     |
+----+------------+
|  1 | Mod

In [5]:
def format_example(row):
    try:
        print(f"\n--- Traitement de la ligne {row.name if hasattr(row, 'name') else '?'} ---")
        print("Données brutes reçues:")
        print(f"Category: {row['category']}")
        print(f"Instruction: {row['instruction']}")
        print(f"Input: {row['input']}")
        print(f"Output: {row['output']}")

        # Validation robuste adaptée au contexte éducatif
        if not isinstance(row['category'], str) or not row['category'].strip():
            raise ValueError("La catégorie doit être une chaîne non vide")

        if not isinstance(row['instruction'], str) or not row['instruction'].strip():
            raise ValueError("L'instruction doit être une chaîne non vide")

        if not isinstance(row['output'], str) or not row['output'].strip():
            raise ValueError("La réponse attendue ne peut pas être vide")

        # Gestion flexible selon le type de requête
        input_content = ""
        if isinstance(row['input'], str) and row['input'].strip():
            input_content = row['input'].strip()
            print("Input valide détecté et conservé")

        # Adaptation dynamique du rôle système
        system_message = (
            f"Tu es un assistant expert de la FSAC spécialisé en {row['category']}. "
            f"Tu assistes les étudiants et professionnels avec bienveillance et précision. "
            f"Ta mission: {row['instruction']}"
        )
        print("\nMessage système généré:")
        print(system_message)

        # Construction contextuelle du message utilisateur
        user_message = input_content if input_content else row['instruction']
        print("\nMessage utilisateur généré:")
        print(user_message)

        result = {
            "messages": [
                {"role": "system", "content": system_message},
                {"role": "user", "content": user_message},
                {"role": "assistant", "content": row["output"]}
            ]
        }

        print("\nFormatage réussi. Résultat final:")
        print(result)
        return result

    except Exception as e:
        error_msg = f"[Ignoré] Ligne {row.name if hasattr(row, 'name') else '?'}: {str(e)}"
        print("\n" + "!"*50)
        print(error_msg)
        print("!"*50 + "\n")
        return None

# Exemple d'application avec affichage des résultats
print("=== DÉBUT DU FORMATAGE DES DONNÉES ===")
formatted_data = []
for idx, row in df.iterrows():
    formatted = format_example(row)
    if formatted:
        formatted_data.append(formatted)
    if idx < 3:  # Affiche les 3 premiers exemples complets
        print("\n" + "="*50)
        print(f"EXEMPLE FORMATÉ {idx+1}:")
        print(formatted if formatted else "IGNORÉ")
        print("="*50 + "\n")

print("\n=== STATISTIQUES FINALES ===")
print(f"Total lignes traitées: {len(df)}")
print(f"Exemples valides: {len(formatted_data)}")
print(f"Exemples ignorés: {len(df) - len(formatted_data)}")
print("=== FORMATAGE TERMINÉ ===")

=== DÉBUT DU FORMATAGE DES DONNÉES ===

--- Traitement de la ligne 0 ---
Données brutes reçues:
Category: Module
Instruction: Listez les modules de chaque semestre. Pour Big Data et Cloud Computing
Input: Quels sont les modules enseignés par semestre De Big Data et Cloud Computing?
Output: "M1 (Fondamentaux Algorithmiques Pour Big Data) M2 (Business Intelligence) M3 (Gestion de projet) M4 (Décision et analyse multicritère) M5 (Management des systèmes d’information) M6 (Programmation web avancée). **Semestre 2** : M7 (Mobile Cloud Computing) M8 (Hadoop Base de Données NoSQL) M9 (Machine Learning) M10 (Web Sémantique) M11 (Techniques de Stockage dans le Cloud) M12 (Anglais). **Semestre 3** : M13 (Data et Text Mining) M14 (Virtualisation et Cloud Computing) M15 (Intelligence Artificielle) M16 (Statistiques décisionnelles) M17 (Data processing avec PIG Hive et Spark) M18 (Programmation Python). **Semestre 4** : M19 à M24 (Stage en entreprise) M2 (Business Intelligence) M3 (Gestion de proje

In [6]:
print(formatted_data)

[{'messages': [{'role': 'system', 'content': 'Tu es un assistant expert de la FSAC spécialisé en Module. Tu assistes les étudiants et professionnels avec bienveillance et précision. Ta mission: Listez les modules de chaque semestre. Pour Big Data et Cloud Computing'}, {'role': 'user', 'content': 'Quels sont les modules enseignés par semestre De Big Data et Cloud Computing?'}, {'role': 'assistant', 'content': '"M1 (Fondamentaux Algorithmiques Pour Big Data) M2 (Business Intelligence) M3 (Gestion de projet) M4 (Décision et analyse multicritère) M5 (Management des systèmes d’information) M6 (Programmation web avancée). **Semestre 2** : M7 (Mobile Cloud Computing) M8 (Hadoop Base de Données NoSQL) M9 (Machine Learning) M10 (Web Sémantique) M11 (Techniques de Stockage dans le Cloud) M12 (Anglais). **Semestre 3** : M13 (Data et Text Mining) M14 (Virtualisation et Cloud Computing) M15 (Intelligence Artificielle) M16 (Statistiques décisionnelles) M17 (Data processing avec PIG Hive et Spark) M1

In [7]:
# 1. Création de formatted_data à partir de votre DataFrame nettoyé (df)
formatted_data = []
for idx, row in df.iterrows():  # df doit être votre DataFrame nettoyé existant
    formatted_example = format_example(row)  # Utilisation de votre fonction existante
    if formatted_example is not None:  # On garde seulement les exemples valides
        formatted_data.append(formatted_example)

# 2. Conversion en DataFrame pandas (comme dans votre demande)
formatted_df = pd.DataFrame(formatted_data)


--- Traitement de la ligne 0 ---
Données brutes reçues:
Category: Module
Instruction: Listez les modules de chaque semestre. Pour Big Data et Cloud Computing
Input: Quels sont les modules enseignés par semestre De Big Data et Cloud Computing?
Output: "M1 (Fondamentaux Algorithmiques Pour Big Data) M2 (Business Intelligence) M3 (Gestion de projet) M4 (Décision et analyse multicritère) M5 (Management des systèmes d’information) M6 (Programmation web avancée). **Semestre 2** : M7 (Mobile Cloud Computing) M8 (Hadoop Base de Données NoSQL) M9 (Machine Learning) M10 (Web Sémantique) M11 (Techniques de Stockage dans le Cloud) M12 (Anglais). **Semestre 3** : M13 (Data et Text Mining) M14 (Virtualisation et Cloud Computing) M15 (Intelligence Artificielle) M16 (Statistiques décisionnelles) M17 (Data processing avec PIG Hive et Spark) M18 (Programmation Python). **Semestre 4** : M19 à M24 (Stage en entreprise) M2 (Business Intelligence) M3 (Gestion de projet) M4 (Décision et analyse multicritère

In [8]:
print(formatted_df)

                                              messages
0    [{'role': 'system', 'content': 'Tu es un assis...
1    [{'role': 'system', 'content': 'Tu es un assis...
2    [{'role': 'system', 'content': 'Tu es un assis...
3    [{'role': 'system', 'content': 'Tu es un assis...
4    [{'role': 'system', 'content': 'Tu es un assis...
..                                                 ...
597  [{'role': 'system', 'content': 'Tu es un assis...
598  [{'role': 'system', 'content': 'Tu es un assis...
599  [{'role': 'system', 'content': 'Tu es un assis...
600  [{'role': 'system', 'content': 'Tu es un assis...
601  [{'role': 'system', 'content': 'Tu es un assis...

[602 rows x 1 columns]


In [9]:
#Extraction de la catégorie depuis les messages pour la stratification
def extract_category(messages):
    try:
        system_content = messages[0]["content"]  # Premier message système
        return system_content.split("spécialisé en ")[1].split(".")[0].strip()
    except:
        return "Unknown"

formatted_df["_category"] = formatted_df["messages"].apply(extract_category)

In [10]:
print(formatted_df)

                                              messages  \
0    [{'role': 'system', 'content': 'Tu es un assis...   
1    [{'role': 'system', 'content': 'Tu es un assis...   
2    [{'role': 'system', 'content': 'Tu es un assis...   
3    [{'role': 'system', 'content': 'Tu es un assis...   
4    [{'role': 'system', 'content': 'Tu es un assis...   
..                                                 ...   
597  [{'role': 'system', 'content': 'Tu es un assis...   
598  [{'role': 'system', 'content': 'Tu es un assis...   
599  [{'role': 'system', 'content': 'Tu es un assis...   
600  [{'role': 'system', 'content': 'Tu es un assis...   
601  [{'role': 'system', 'content': 'Tu es un assis...   

                   _category  
0                     Module  
1                     Module  
2                     Module  
3                     Module  
4                     Module  
..                       ...  
597     Critères d'admission  
598     Critères d'admission  
599     Critères d'admis

In [11]:
#split startifié
def stratified_split(df):
    # Premier split : 85% train_val, 15% test
    train_val, test = train_test_split(
        df,
        test_size=0.15,
        stratify=df["_category"],
        random_state=42
    )

    # Deuxième split : ~17.65% de l'original pour val (soit 15% du total)
    train, val = train_test_split(
        train_val,
        test_size=0.1765,
        stratify=train_val["_category"],
        random_state=42
    )

    return train, val, test

train_df, val_df, test_df = stratified_split(formatted_df)

In [12]:
print(test_df)

                                              messages  \
493  [{'role': 'system', 'content': 'Tu es un assis...   
373  [{'role': 'system', 'content': 'Tu es un assis...   
227  [{'role': 'system', 'content': 'Tu es un assis...   
317  [{'role': 'system', 'content': 'Tu es un assis...   
221  [{'role': 'system', 'content': 'Tu es un assis...   
..                                                 ...   
368  [{'role': 'system', 'content': 'Tu es un assis...   
355  [{'role': 'system', 'content': 'Tu es un assis...   
552  [{'role': 'system', 'content': 'Tu es un assis...   
196  [{'role': 'system', 'content': 'Tu es un assis...   
482  [{'role': 'system', 'content': 'Tu es un assis...   

                         _category  
493  Information sur l'emplacement  
373                      Débouchés  
227                         Atouts  
317                      Débouchés  
221                     Partenaire  
..                             ...  
368                      Débouchés  
355    

In [13]:
# Conversion finale avec vérification
print("\n=== CONVERSION FINALE ===")

# Création du DatasetDict
final_dataset = DatasetDict({
    "train": Dataset.from_pandas(train_df.drop(columns=["_category"])),
    "validation": Dataset.from_pandas(val_df.drop(columns=["_category"])),
    "test": Dataset.from_pandas(test_df.drop(columns=["_category"]))
})

# Affichage des statistiques
print("\n🔍 Statistiques des datasets:")
print(f"Train: {len(final_dataset['train'])} exemples")
print(f"Validation: {len(final_dataset['validation'])} exemples")
print(f"Test: {len(final_dataset['test'])} exemples")

# Vérification du format
print("\n✅ Structure des données:")
print(final_dataset)

# Affichage d'un exemple de chaque split
print("\n📝 Exemples de données:")
for split in ['train', 'validation', 'test']:
    print(f"\n--- {split.upper()} ---")
    example = final_dataset[split][0]  # Premier exemple
    for key, value in example.items():
        print(f"{key}: {str(value)[:100]}{'...' if len(str(value)) > 100 else ''}")

# Vérification des colonnes
print("\n🧐 Colonnes présentes:")
for split in final_dataset.keys():
    print(f"{split}: {final_dataset[split].column_names}")

print("\n=== CONVERSION TERMINÉE AVEC SUCCÈS ===")


=== CONVERSION FINALE ===

🔍 Statistiques des datasets:
Train: 420 exemples
Validation: 91 exemples
Test: 91 exemples

✅ Structure des données:
DatasetDict({
    train: Dataset({
        features: ['messages', '__index_level_0__'],
        num_rows: 420
    })
    validation: Dataset({
        features: ['messages', '__index_level_0__'],
        num_rows: 91
    })
    test: Dataset({
        features: ['messages', '__index_level_0__'],
        num_rows: 91
    })
})

📝 Exemples de données:

--- TRAIN ---
messages: [{'content': 'Tu es un assistant expert de la FSAC spécialisé en Objectif. Tu assistes les étudiants...
__index_level_0__: 212

--- VALIDATION ---
messages: [{'content': "Tu es un assistant expert de la FSAC spécialisé en Débouchés. Tu assistes les étudiant...
__index_level_0__: 336

--- TEST ---
messages: [{'content': "Tu es un assistant expert de la FSAC spécialisé en Information sur l'emplacement. Tu a...
__index_level_0__: 493

🧐 Colonnes présentes:
train: ['messages', 

In [15]:
#verification
def analyze_dataset(dataset):
    print(f"\n{'='*40}\nAnalyse du dataset\n{'='*40}")

    # Stats globales
    total = sum(len(dataset[split]) for split in dataset)
    print(f"\nRépartition globale ({total} exemples):")
    for split in dataset:
        pct = len(dataset[split]) / total * 100
        print(f"- {split}: {len(dataset[split])} exemples ({pct:.1f}%)")

    # Distribution des catégories
    def get_category_distribution(split):
        categories = []
        for example in dataset[split]:
            cat = extract_category(example["messages"])
            categories.append(cat)
        return pd.Series(categories).value_counts(normalize=True) * 100

    print("\nDistribution des catégories (%):")
    for split in dataset:
        print(f"\n{split.capitalize()}:")
        print(get_category_distribution(split).to_string())

analyze_dataset(final_dataset)

# 6. Exemple de vérification détaillée
print("\nExemple détaillé (train[0]):")
sample = final_dataset["train"][0]
for message in sample["messages"]:
    print(f"\n{message['role'].upper()}:\n{message['content']}")


Analyse du dataset

Répartition globale (602 exemples):
- train: 420 exemples (69.8%)
- validation: 91 exemples (15.1%)
- test: 91 exemples (15.1%)

Distribution des catégories (%):

Train:
Débouchés                        18.095238
Module                           15.238095
Atouts                           14.285714
competences visees               13.809524
Coordonnées                      10.714286
Objectif                          5.238095
Frais de la formation             5.238095
Questions Ouvertes                3.333333
Questions Générales               2.857143
Excuses                           2.857143
Salutations et Adieux             1.904762
objectif                          1.666667
Partenaire                        1.428571
Critères d'admission              1.190476
Information sur l'emplacement     1.190476
Processus d'inscription           0.476190
Questions et Réponses             0.476190

Validation:
Débouchés                        18.681319
Module                

In [16]:
print(final_dataset)

DatasetDict({
    train: Dataset({
        features: ['messages', '__index_level_0__'],
        num_rows: 420
    })
    validation: Dataset({
        features: ['messages', '__index_level_0__'],
        num_rows: 91
    })
    test: Dataset({
        features: ['messages', '__index_level_0__'],
        num_rows: 91
    })
})


In [19]:
!pip install bitsandbytes

In [22]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

model = AutoModelForCausalLM.from_pretrained(
    base_model,
    device_map="auto",
    trust_remote_code=True,
)

# 4. Prepare for k-bit LoRA Training
model.config.use_cache = False
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

# 5. Attach LoRA Adapter
peft_config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, peft_config)

/usr/local/lib/python3.11/dist-packages/auto_gptq/nn_modules/triton_utils/kernels.py:410: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/usr/local/lib/python3.11/dist-packages/auto_gptq/nn_modules/triton_utils/kernels.py:418: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd
/usr/local/lib/python3.11/dist-packages/auto_gptq/nn_modules/triton_utils/kernels.py:461: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd(cast_inputs=torch.float16)
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.
Some weights of the model checkpoint at TheBloke/zephyr-7B-beta-GPTQ were not used when initializing MistralForCausalLM: ['model.layers.0.m

In [25]:
# 6. Tokenization Function for Dataset
def tokenize_function(examples):
    inputs = tokenizer(
        examples["input"],
        truncation=True,
        padding="max_length",
        max_length=512
    )
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["label"],
            truncation=True,
            padding="max_length",
            max_length=512
        )
    inputs["labels"] = labels["input_ids"]
    return inputs

# 7. Tokenize Dataset
tokenized_dataset = final_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/420 [00:00<?, ? examples/s]

KeyError: 'input'

In [26]:
from datasets import DatasetDict

# 1) Define how to turn a list of messages into a single string
def build_chat_input(messages):
    # each turn: e.g. "<|system|> You are ...<|user|> Bonjour ...<|assistant|> Voici ..."
    s = ""
    for turn in messages:
        role = turn["role"]
        content = turn["content"]
        s += f"<|{role}|>{content}"
    s += "<|endoftext|>"
    return s
tokenizer = AutoTokenizer.from_pretrained(
    base_model,
    trust_remote_code=True,
    padding_side="right",
    truncation_side="right"
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# 2) Batched tokenization function
def tokenize_function(examples):
    # examples["messages"] is a list of lists of dicts
    inputs = [build_chat_input(msgs) for msgs in examples["messages"]]
    # tokenize + pad/truncate
    tokenized = tokenizer(
        inputs,
        truncation=True,
        max_length=2048,
        padding="max_length",
    )
    # for causal-LM fine-tuning we typically set labels = input_ids
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

# 3) The wrapper that maps + sanity-checks
def process_dataset(raw_dataset: DatasetDict) -> DatasetDict:
    """
    raw_dataset: a DatasetDict with splits like "train", "validation", each example having "messages" field.
    Returns a new DatasetDict where each split has only input_ids, attention_mask, labels, etc.
    """
    try:
        # map over all splits
        tokenized = raw_dataset.map(
            tokenize_function,
            batched=True,
            batch_size=8,
            remove_columns=raw_dataset["train"].column_names,
            load_from_cache_file=False
        )

        # quick sanity check on one example
        sample = tokenized["train"][0]
        assert "input_ids" in sample
        assert "attention_mask" in sample
        assert "labels" in sample
        assert len(sample["input_ids"]) <= 2048

        return tokenized

    except Exception as e:
        print(f"❌ Échec tokenization: {e}\n")
        print("=== Debugging first 3 train examples ===")
        for i in range(min(3, len(raw_dataset["train"]))):
            msgs = raw_dataset["train"][i]["messages"]
            try:
                _ = tokenize_function({"messages": [msgs]})
                print(f"✔ Exemple #{i} tokenisé avec succès")
            except Exception as e2:
                print(f"❌ Exemple #{i} a échoué:")
                for turn in msgs:
                    role = turn["role"].upper()
                    content = turn["content"]
                    preview = content[:200] + ("..." if len(content)>200 else "")
                    print(f"  • {role} ({len(content)} chars): {preview}")
        raise

# ── Usage ───────────────────────────────────────────────────────────────
# Assuming `raw_dataset` is your DatasetDict loaded from JSON/CSV/etc.
tokenized_dataset = process_dataset(final_dataset)

# Now hand `tokenized_dataset["train"]` and `["validation"]` to your trainer


Map:   0%|          | 0/420 [00:00<?, ? examples/s]

Map:   0%|          | 0/91 [00:00<?, ? examples/s]

Map:   0%|          | 0/91 [00:00<?, ? examples/s]

In [30]:
OUTPUT_DIR='zephyr-finetuning'
OPTIMIZER = "paged_adamw_32bit"

training_arguments = TrainingArguments(
                                        output_dir=OUTPUT_DIR,
                                        per_device_train_batch_size=8,
                                        gradient_accumulation_steps=1,
                                        optim=OPTIMIZER,
                                        learning_rate=2e-4,
                                        lr_scheduler_type="cosine",
                                        save_strategy="epoch",
                                        logging_steps=50,
                                        num_train_epochs=1,
                                        max_steps=100,
                                        fp16=True,
                                        eval_strategy="steps",
                                        eval_steps=50,
                                        push_to_hub=False,
                                        report_to="none" )
#finetuning
trainer = SFTTrainer(
                        model=model,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
                        peft_config=peft_config,
                        args=training_arguments,
                    )

trainer.train()

Truncating train dataset:   0%|          | 0/420 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/91 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss,Validation Loss
50,0.457500,0.098979
100,0.089000,0.089563


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


TrainOutput(global_step=100, training_loss=0.2732433080673218, metrics={'train_runtime': 2422.7666, 'train_samples_per_second': 0.33, 'train_steps_per_second': 0.041, 'total_flos': 708992431030272.0, 'train_loss': 0.2732433080673218})

In [ ]:
from transformers import AutoTokenizer, GenerationConfig
from peft import AutoPeftModelForCausalLM
import torch

# Load tokenizer and model
model_path = "/content/zephyr-finetuning/checkpoint-100"

tokenizer = AutoTokenizer.from_pretrained(model_path)
tokenizer.pad_token = tokenizer.eos_token

model = AutoPeftModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.float16,
    device_map="auto"
)
model.eval()

# Define chat context template
system_prompt = "<|system|>\nTu es un assistant expert de la FSAC spécialisé en Objectif. Tu assistes les étudiants et professionnels avec bienveillance et précision.\n"

def build_prompt(chat_history):
    prompt = system_prompt
    for user_msg, bot_msg in chat_history:
        prompt += f"<|user|>\n{user_msg}\n<|assistant|>\n{bot_msg}\n"
    return prompt

# Chat loop
chat_history = []

print("💬 Chat with your fine-tuned Zephyr model! (type 'exit' to quit)\n")

while True:
    user_input = input("🧑 You: ")
    if user_input.lower() == "exit":
        break

    # Build prompt with history
    prompt = build_prompt(chat_history + [(user_input, "")])

    # Tokenize and generate
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to("cuda")

    generation_config = GenerationConfig(
        do_sample=True,
        temperature=0.7,
        top_k=50,
        max_new_tokens=500,
        pad_token_id=tokenizer.eos_token_id
    )

    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            generation_config=generation_config
        )

    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    # Extract only the latest assistant response
    assistant_reply = output_text.split("<|assistant|>\n")[-1].strip()
    print(f"🤖 Bot: {assistant_reply}\n")

    # Save turn in history
    chat_history.append((user_input, assistant_reply))


/usr/local/lib/python3.11/dist-packages/accelerate/utils/modeling.py:1569: UserWarning: Current model requires 567337088 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(
Some weights of the model checkpoint at TheBloke/zephyr-7B-beta-GPTQ were not used when initializing MistralForCausalLM: ['model.layers.0.mlp.down_proj.bias', 'model.layers.0.mlp.gate_proj.bias', 'model.layers.0.mlp.up_proj.bias', 'model.layers.0.self_attn.k_proj.bias', 'model.layers.0.self_attn.o_proj.bias', 'model.layers.0.self_attn.q_proj.bias', 'model.layers.0.self_attn.v_proj.bias', 'model.layers.1.mlp.down_proj.bias', 'model.layers.1.mlp.gate_proj.bias', 'model.layers.1.mlp.up_proj.bias', 'model.layers.1.self_attn.k_proj.bias', 'model.layers.1.self_attn.o_proj.bias', 'model.layers.1.self_attn.q_proj.bias', 'model.layers.1.self_attn.v_proj.bias', 'model.layers.10.mlp.down_proj

💬 Chat with your fine-tuned Zephyr model! (type 'exit' to quit)

🧑 You: Quels sont les modules enseignés en Semestre 4 de Big Data et Cloud Computing?


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


🤖 Bot: M15 : Big Data et Cloud Computing (BDC) Semestre 4.<|endoftext|>

🧑 You: Listez les modules du Semestre 2. Pour Big Data et Cloud Computing
🤖 Bot: M16 : Infrastructure Cloud (Amazon Web Services)
M17 : Big Data Frameworks (Hadoop)
M18 : Big Data Architecture et Déploiements
M19 : Big Data Security et Gestion des Données Sensibles
M20 : Big Data et l'Intelligence Artificielle (IA)
<|endoftext|>



In [ ]:
# 3. Split stratifié via sklearn (nécessite conversion temporaire en pandas)
formatted_df = formatted_dataset.to_pandas()
formatted_df["_category"] = df["category"]  # Colonne temporaire pour stratification

# Split 70%/15%/15%
train_val, test = train_test_split(
    formatted_df,
    test_size=0.15,
    stratify=formatted_df["_category"],
    random_state=42
)
train, val = train_test_split(
    train_val,
    test_size=0.176,  # 15% du total
    stratify=train_val["_category"],
    random_state=42
)

# 4. Conversion finale en DatasetDict
final_dataset = DatasetDict({
    "train": Dataset.from_pandas(train.drop(columns=["_category"])),
    "validation": Dataset.from_pandas(val.drop(columns=["_category"])),
    "test": Dataset.from_pandas(test.drop(columns=["_category"]))
})

# 4. Vérification
print(f"Train: {len(final_dataset['train'])} exemples")
print(f"Validation: {len(final_dataset['validation'])} exemples")
print(f"Test: {len(final_dataset['test'])} exemples\n")

print("Exemple train[0]:", final_dataset["train"][0])

Train: 421 exemples
Validation: 90 exemples
Test: 91 exemples

Exemple train[0]: {'prompt': '[Catégorie: Questions Générales]\n[Instruction: Demander la destination de vacances préférée.]\nInput: Quelle est votre destination de vacances préférée ?', 'label': 'Les vacances sont spéciales. Où aimez-vous aller ?', '__index_level_0__': 467}


In [ ]:
print(formatted_df)

                                                prompt  \
0    [Catégorie: Module]\n[Instruction: Listez les ...   
1    [Catégorie: Module]\n[Instruction: Quel est l'...   
2    [Catégorie: Module]\n[Instruction: Listez les ...   
3    [Catégorie: Module]\n[Instruction: Listez les ...   
4    [Catégorie: Module]\n[Instruction: Listez les ...   
..                                                 ...   
597  [Catégorie: Critères d'admission]\n[Instructio...   
598  [Catégorie: Critères d'admission]\n[Instructio...   
599  [Catégorie: Critères d'admission]\n[Instructio...   
600  [Catégorie: Processus d'inscription]\n[Instruc...   
601  [Catégorie: Processus d'inscription]\n[Instruc...   

                                                 label  \
0     "M1 (Fondamentaux Algorithmiques Pour Big Dat...   
1    Stage en entreprise pour appliquer les connais...   
2    M1 : Fondamentaux Algorithmiques Pour Big Data...   
3    M7 : Mobile Cloud Computing, M8 : Hadoop Base ...   
4    M13 : Da

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
from trl import SFTTrainer

# Base model name (replace with your actual model)
base_model

# 1. Tokenizer Configuration
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.add_bos_token = True  # You might want to explicitly add the BOS token too

# 2. Quantization Configuration for GPTQ (Low-Bit Training)
bnb_config = GPTQConfig(bits=4,
                        disable_exllama=True,
                        device_map="auto",
                        use_cache=False,
                        lora_r=16,
                        lora_alpha=16,
                        tokenizer=tokenizer
                        )

# 3. Load Quantized Model
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config,
)

# Enable gradient checkpointing for reduced memory usage
model.config.use_cache = False
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()

# Prepare the model for k-bit training (specific to your setup)
model = prepare_model_for_kbit_training(model)

# 4. Tokenization Function for Dataset
def tokenize_function(examples):
    return tokenizer(
        examples["prompt"],
        text_target=examples["label"],
        max_length=512,
        truncation=True,
        padding="max_length",
        return_tensors="pt"
    )

# Assuming `final_dataset` is your dataset with 'prompt' and 'label' columns
tokenized_dataset = final_dataset.map(tokenize_function, batched=True)

# 5. Tokenization Verification
print("\nTokenization vérifiée sur tous les splits :")
for split in ["train", "validation", "test"]:
    print(f"\n{split.upper()} split:")
    print(f"Premiers tokens input_ids : {tokenized_dataset[split][0]['input_ids'][:5]}")
    print(f"Premiers tokens labels : {tokenized_dataset[split][0]['labels'][:5]}")
    print(f"Exemple complet :")
    print(f"Prompt original : {final_dataset[split][0]['prompt']}")
    print(f"Label original : {final_dataset[split][0]['label']}")
    print(f"Tokens du prompt : {tokenizer.convert_ids_to_tokens(tokenized_dataset[split][0]['input_ids'][:10])}")
    print(f"Tokens du label : {tokenizer.convert_ids_to_tokens(tokenized_dataset[split][0]['labels'][:10])}")

# Output model structure for verification
print("\nModel structure after preparation:")
print(model)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/quantizers/auto.py:212: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.However, loading attributes (e.g. ['backend', 'use_cuda_fp16', 'use_exllama', 'max_input_length', 'exllama_config']) will be overwritten with the one you passed to `from_pretrained`. 

Map:   0%|          | 0/421 [00:00<?, ? examples/s]

Map:   0%|          | 0/90 [00:00<?, ? examples/s]

Map:   0%|          | 0/91 [00:00<?, ? examples/s]


Tokenization vérifiée sur tous les splits :

TRAIN split:
Premiers tokens input_ids : [1, 733, 19962, 28797, 4553]
Premiers tokens labels : [1, 4573, 7255, 2233, 5497]
Exemple complet :
Prompt original : [Catégorie: Questions Générales]
[Instruction: Demander la destination de vacances préférée.]
Input: Quelle est votre destination de vacances préférée ?
Label original : Les vacances sont spéciales. Où aimez-vous aller ?
Tokens du prompt : ['<s>', '▁[', 'Cat', 'é', 'gor', 'ie', ':', '▁Questions', '▁G', 'én']
Tokens du label : ['<s>', '▁Les', '▁vac', 'ances', '▁sont', '▁spé', 'cial', 'es', '.', '▁O']

VALIDATION split:
Premiers tokens input_ids : [1, 733, 19962, 28797, 4553]
Premiers tokens labels : [1, 550, 607, 8631, 20650]
Exemple complet :
Prompt original : [Catégorie: Coordonnées]
[Instruction: Donner les coordonnées du responsable.]
Input: Comment contacter le responsable de la formation Electronique Electrotechnique Automatique et Informatique Industrielle (EEAII) ?
Label origin

In [ ]:
# preparing model for finetuning
peft_config = LoraConfig(
                            r=16,
                            lora_alpha=16,
                            lora_dropout=0.05,
                            bias="none",
                            task_type="CAUSAL_LM",
                            target_modules=["q_proj", "v_proj"]
                        )

model = get_peft_model(model, peft_config)
print(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=2)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (k_proj): QuantLinear()
              (o_proj): QuantLinear()
              (q_proj): lora.GPTQLoraLinear(
                (base_layer): QuantLinear()
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): 

In [ ]:
OUTPUT_DIR='zephyr-finetuning'
OPTIMIZER = "paged_adamw_32bit"

training_arguments = TrainingArguments(
                                        output_dir=OUTPUT_DIR,
                                        per_device_train_batch_size=8,
                                        gradient_accumulation_steps=1,
                                        optim=OPTIMIZER,
                                        learning_rate=2e-4,
                                        lr_scheduler_type="cosine",
                                        save_strategy="epoch",
                                        logging_steps=50,
                                        num_train_epochs=1,
                                        max_steps=100,
                                        fp16=True,
                                        eval_strategy="steps",
                                        eval_steps=50,
                                        push_to_hub=False,
                                        report_to="none" )
#finetuning
trainer = SFTTrainer(
                        model=model,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
                        peft_config=peft_config,
                        args=training_arguments,
                    )

trainer.train()

Truncating train dataset:   0%|          | 0/421 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/90 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss,Validation Loss
50,0.409600,0.108821
100,0.090100,0.095483


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


TrainOutput(global_step=100, training_loss=0.24984790802001952, metrics={'train_runtime': 1240.0278, 'train_samples_per_second': 0.645, 'train_steps_per_second': 0.081, 'total_flos': 338254003961856.0, 'train_loss': 0.24984790802001952})

In [ ]:
#data preprocessing for the sample input data
def input_data_preprocessing(example):

    processed_example = "<|system|>\n Tu es un assistant expert de la FSAC spécialisé en Objectif. Tu assistes les étudiants et professionnels avec bienveillance et précision. Ta mission: Listez les objectifs principaux de la formation en Modélisation et Simulation en Mécanique \n<|user|>\n" + example["instruction"] + "\n<|assistant|>\n"

    return processed_example
input_string = input_data_preprocessing(
    {
        "instruction": "Objectifs Modélisation et Simulation en Mécanique",
    }
)


tokenizer = AutoTokenizer.from_pretrained("/content/zephyr-finetuning/checkpoint-100")


model = AutoPeftModelForCausalLM.from_pretrained(
    "/content/zephyr-finetuning/checkpoint-100",
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="cuda")

inputs = tokenizer(input_string, return_tensors="pt").to("cuda")

generation_config = GenerationConfig(
    do_sample=True,
    top_k=1,
    temperature=0.1,
    max_new_tokens=256,
    pad_token_id=tokenizer.eos_token_id
)

outputs = model.generate(**inputs, generation_config=generation_config)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Some weights of the model checkpoint at TheBloke/zephyr-7B-beta-GPTQ were not used when initializing MistralForCausalLM: ['model.layers.0.mlp.down_proj.bias', 'model.layers.0.mlp.gate_proj.bias', 'model.layers.0.mlp.up_proj.bias', 'model.layers.0.self_attn.k_proj.bias', 'model.layers.0.self_attn.o_proj.bias', 'model.layers.0.self_attn.q_proj.bias', 'model.layers.0.self_attn.v_proj.bias', 'model.layers.1.mlp.down_proj.bias', 'model.layers.1.mlp.gate_proj.bias', 'model.layers.1.mlp.up_proj.bias', 'model.layers.1.self_attn.k_proj.bias', 'model.layers.1.self_attn.o_proj.bias', 'model.layers.1.self_attn.q_proj.bias', 'model.layers.1.self_attn.v_proj.bias', 'model.layers.10.mlp.down_proj.bias', 'model.layers.10.mlp.gate_proj.bias', 'model.layers.10.mlp.up_proj.bias', 'model.layers.10.self_attn.k_proj.bias', 'model.layers.10.self_attn.o_proj.bias', 'model.layers.10.self_attn.q_proj.bias', 'model.layers.10.self_attn.v_proj.bias', 'model.layers.11.mlp.down_proj.bias', 'model.layers.11.mlp.gate_

<|system|>
 Tu es un assistant expert de la FSAC spécialisé en Objectif. Tu assistes les étudiants et professionnels avec bienveillance et précision. Ta mission: Listez les objectifs principaux de la formation en Modélisation et Simulation en Mécanique 
<|user|>
Objectifs Modélisation et Simulation en Mécanique
<|assistant|>
1. Acquérir les connaissances théoriques et pratiques en modélisation et simulation en mécanique.
2. Apprendre à utiliser les logiciels de modélisation et simulation en mécanique.
3. Développer les compétences en analyse et conception de systèmes mécaniques complexes.
4. Apprendre à utiliser les outils de simulation pour la conception et la validation de systèmes mécaniques.
5. Acquérir les compétences en analyse et conception de systèmes mécaniques pour l'industrie aéronautique et spatiale.
6. Apprendre à utiliser les outils de simulation pour la conception et la validation de systèmes mécaniques pour l'industrie automobile et automobile.
7. Acquérir les compétenc

In [ ]:
from transformers import AutoTokenizer, GenerationConfig
from peft import AutoPeftModelForCausalLM
import torch

# Load tokenizer and model
model_path = "/content/zephyr-finetuning/checkpoint-61"

tokenizer = AutoTokenizer.from_pretrained(model_path)
tokenizer.pad_token = tokenizer.eos_token

model = AutoPeftModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.float16,
    device_map="auto"
)
model.eval()

# Define chat context template
system_prompt = "<|system|>\nTu es un assistant expert de la FSAC spécialisé en Objectif. Tu assistes les étudiants et professionnels avec bienveillance et précision.\n"

def build_prompt(chat_history):
    prompt = system_prompt
    for user_msg, bot_msg in chat_history:
        prompt += f"<|user|>\n{user_msg}\n<|assistant|>\n{bot_msg}\n"
    return prompt

# Chat loop
chat_history = []

print("💬 Chat with your fine-tuned Zephyr model! (type 'exit' to quit)\n")

while True:
    user_input = input("🧑 You: ")
    if user_input.lower() == "exit":
        break

    # Build prompt with history
    prompt = build_prompt(chat_history + [(user_input, "")])

    # Tokenize and generate
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to("cuda")

    generation_config = GenerationConfig(
        do_sample=True,
        temperature=0.7,
        top_k=50,
        max_new_tokens=256,
        pad_token_id=tokenizer.eos_token_id
    )

    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            generation_config=generation_config
        )

    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    # Extract only the latest assistant response
    assistant_reply = output_text.split("<|assistant|>\n")[-1].strip()
    print(f"🤖 Bot: {assistant_reply}\n")

    # Save turn in history
    chat_history.append((user_input, assistant_reply))


/usr/local/lib/python3.11/dist-packages/accelerate/utils/modeling.py:1569: UserWarning: Current model requires 1475076224 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(
Some weights of the model checkpoint at TheBloke/zephyr-7B-beta-GPTQ were not used when initializing MistralForCausalLM: ['model.layers.0.mlp.down_proj.bias', 'model.layers.0.mlp.gate_proj.bias', 'model.layers.0.mlp.up_proj.bias', 'model.layers.0.self_attn.k_proj.bias', 'model.layers.0.self_attn.o_proj.bias', 'model.layers.0.self_attn.q_proj.bias', 'model.layers.0.self_attn.v_proj.bias', 'model.layers.1.mlp.down_proj.bias', 'model.layers.1.mlp.gate_proj.bias', 'model.layers.1.mlp.up_proj.bias', 'model.layers.1.self_attn.k_proj.bias', 'model.layers.1.self_attn.o_proj.bias', 'model.layers.1.self_attn.q_proj.bias', 'model.layers.1.self_attn.v_proj.bias', 'model.layers.10.mlp.down_pro

💬 Chat with your fine-tuned Zephyr model! (type 'exit' to quit)

🧑 You: Donner le prix de la formation Administration Avancée des Systèmes et RéseauxInformatiques (AASRI).


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


🤖 Bot: La formation coûte 12000 DH.

🧑 You: Quel est le prix de la formation Génie Energie Solaire et Eolienne (GESE) ?
🤖 Bot: La formation coûte 12000 DH.

🧑 You: Quels sont les modules enseignés en Semestre 1 de Cybersécurité et Cyberdéfense?
🤖 Bot: M1 : Cybersécurité, Cyberdéfense, Cybercriminalité, M2 : Systèmes d'information, Architecture et Systèmes d'intégration, M3 : Cloud Computing, M4 : Cybersécurité et Cyberdéfense, M5 : Cybersécurité et Cyberdéfense



KeyboardInterrupt: Interrupted by user

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
